In [14]:
import bioframe as bf
import pandas as pd
import copy
import numpy as np
from schema import DFSchema
from df_loader import CrisprDFLoader, PredDFLoader
from overlaps import (
    read_overlaps_from_file, merge_multiple_predictions, compute_crispr_overlaps
)
# pd.set_option('display.max_rows', 1000) 

In [15]:
CRISPR_FILENAME = "/oak/stanford/groups/engreitz/Projects/Benchmarking/CRISPR_data/EPCrisprBenchmark_ensemble_data_GRCh38.tsv.gz"
PRED_FILENAME = "/oak/stanford/groups/engreitz/Users/atan5133/abc_run_comparisons/results_no_qnorm_08_28_dev/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
OVERLAP_FILENAME = "crispr_pred_overlaps_noqnorm.csv"
ABC_THRESHOLD = 0.02
TSS_REF_FILE = "resources/genome_annotations/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.TSS500bp.bed"
TARGET_GENE = "GATA1"

In [16]:
pred_df = PredDFLoader(PRED_FILENAME, TSS_REF_FILE, ABC_THRESHOLD).load()
crispr_df = CrisprDFLoader(CRISPR_FILENAME, TSS_REF_FILE).load()
# overlap_df = compute_crispr_overlaps(
#     crispr_df, pred_df
# )

In [17]:
overlap_df = read_overlaps_from_file(OVERLAP_FILENAME)

In [18]:
df = merge_multiple_predictions(overlap_df, ABC_THRESHOLD)

In [19]:
def format_distance(bp):
    kb = bp / 1000
    mb = kb / 1000

    if mb >= 1:
        return f"{mb:.2f} Mbp"
    elif kb >= 1:
        return f"{kb:.2f} Kbp"
    else:
        return f"{bp} bp"

In [20]:
gene_df = df[df["TargetGene_crispr"] == TARGET_GENE]
# pretty print distance
gene_df.loc[:,'distance_pred'] = gene_df['distance_pred'].apply(format_distance)

In [26]:
# Sorted by top 10 contact
gene_df_sorted = gene_df.sort_values(by="hic_contact_pl_scaled_adj_pred", ascending=False)
columns_pred = ['name', 'distance', 'activity_base', 'hic_contact', 'ABC.Score', 'IsSignificant', 'hic_contact_pl_scaled_adj','ABC.Score.Numerator', 'normalized_dhs', 'isSelfPromoter', 'powerlaw_contact', 'powerlaw_contact_reference','hic_contact_pl_scaled']
columns = ["dataset_crispr", "start_crispr", "end_crispr", "pValueAdjusted_crispr", "EffectSize_crispr", "Regulated_crispr"] + [col+'_pred' for col in columns_pred]
gene_df_sorted[:10][columns]

,dataset_crispr,start_crispr,end_crispr,pValueAdjusted_crispr,EffectSize_crispr,Regulated_crispr,name_pred,distance_pred,activity_base_pred,hic_contact_pred,ABC.Score_pred,IsSignificant_pred,hic_contact_pl_scaled_adj_pred,ABC.Score.Numerator_pred,normalized_dhs_pred,isSelfPromoter_pred,powerlaw_contact_pred,powerlaw_contact_reference_pred,hic_contact_pl_scaled_pred
9549,FlowFISH_K562,48800621,48800667,NaN,-0.330000,True,intergenic|chrX:48800213-48801055,14.06 Kbp,42.260009,0.023835,0.080040,True,0.087241,3.686788,43.184744,False,0.017052,0.050214,0.070189
9590,FlowFISH_K562,49155148,49156608,0.858038,0.003343,False,intergenic|chrX:49155190-49156881,369.46 Kbp,39.711592,0.001245,0.005457,False,0.006329,0.251341,34.215605,False,0.000640,0.002923,0.005689
9600,FlowFISH_K562,49190908,49191428,0.966121,-0.020442,False,genic|chrX:49190732-49192107,404.85 Kbp,34.662327,0.000509,0.002212,False,0.002940,0.101893,66.770257,False,0.000584,0.002699,0.002356
9455,FlowFISH_K562,47483806,47484326,0.966121,-0.005510,False,promoter|chrX:47482358-47484263,1.30 Mbp,33.062391,0.000000,0.000129,False,0.000180,0.005962,22.339800,False,0.000180,0.000976,0.000000
9561,FlowFISH_K562,48919808,48920708,0.966121,-0.006855,False,promoter|chrX:48918824-48920454,133.07 Kbp,30.485136,0.002869,0.008744,False,0.013212,0.402767,35.046080,False,0.001784,0.007107,0.011428
9562,FlowFISH_K562,48936994,48938034,0.706716,-0.039583,False,genic|chrX:48936943-48937941,150.87 Kbp,29.362822,0.003511,0.010071,False,0.015799,0.463899,21.675419,False,0.001573,0.006371,0.014226
9538,FlowFISH_K562,48737018,48737918,0.953750,-0.029144,False,intergenic|chrX:48736965-48737922,49.13 Kbp,28.737405,0.009643,0.023990,True,0.038452,1.105007,34.879985,False,0.004853,0.016910,0.033598
9594,FlowFISH_K562,49167166,49167746,0.754057,-0.038578,False,genic|chrX:49166324-49167829,380.50 Kbp,24.772761,0.001329,0.003614,False,0.006720,0.166465,12.623233,False,0.000621,0.002849,0.006099
9593,FlowFISH_K562,49166366,49166866,0.966121,0.004620,False,genic|chrX:49166324-49167829,380.50 Kbp,24.772761,0.001329,0.003614,False,0.006720,0.166465,12.623233,False,0.000621,0.002849,0.006099
9563,FlowFISH_K562,48940454,48940954,0.953750,-0.034758,False,genic|chrX:48940439-48941270,154.28 Kbp,24.577262,0.004568,0.010724,False,0.020099,0.493985,23.917704,False,0.001538,0.006249,0.018562


In [ ]:
# top 15 effect sizes
gene_df.sort_values(by="EffectSize_crispr", ascending=False)[columns][:15]

In [22]:
# Total Significant enhancers: 14
print(len(gene_df[gene_df["Regulated_crispr"]]))

# Total predicted signficant enhancers: 6
print(len(gene_df[gene_df["IsSignificant_pred"]]))

# This shows that normalizing across E-G pairs doesn't work for cases like this.
# We miss out on a lot of predictions

3
3


In [25]:
gene_df[gene_df["Regulated_crispr"]]

,dataset_crispr,chrom_crispr,start_crispr,end_crispr,name_crispr,EffectSize_crispr,chrTSS_crispr,TargetGeneTSS_crispr,endTSS_crispr,TargetGene_crispr,...,hic_contact_pl_scaled_adj_pred,ABC.Score.Numerator_pred,ABC.Score_pred,powerlaw.Score.Numerator_pred,powerlaw.Score_pred,CellType_pred,hic_contact_squared_pred,DistanceToTSS_pred,IsSignificant_pred,from_mult_pred
9546,Gasperini2019,chrX,48782818,48783318,GATA1|chrX:48641226-48641726:.,-0.308856,chrX,48786323,48786555.0,GATA1,...,0.248492,3.699880,0.080324,1.837878,0.085530,K562,0.006111,3743.0,True,False
9547,FlowFISH_K562,chrX,48784617,48785277,GATA1|chrX:48643025-48643685:*,-0.187127,chrX,48786323,48786555.0,GATA1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
9549,FlowFISH_K562,chrX,48800621,48800667,GATA1|chrX:48659028-48659074:*,-0.330000,chrX,48786323,48786555.0,GATA1,...,0.087241,3.686788,0.080040,2.122023,0.098753,K562,0.000568,-13640.0,True,False
